In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
#package imports
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import seaborn as sns
import tensorflow



from nltk.corpus import stopwords
from plotly.offline import iplot
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from transformers import BertConfig, BertTokenizerFast, TFBertModel

In [54]:
# Recreate the exact same model, including its weights and the optimizer
bert_model = tensorflow.keras.models.load_model('/content/drive/MyDrive/Final_pipeline/my_model.h5')

# Show the model architecture
bert_model.summary()

Model: "BERT_MultiLabel_MultiClass"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 100)]             0         
                                                                 
 bert (TFBertMainLayer)      TFBaseModelOutputWithPoo  109482240 
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             100, 768),                          
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                        

In [55]:

import spacy
import re
import nltk
#en_core_web_sm: this package must be the newest version, currently 3.5.0   (by 2023.04.02)
!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0.tar.gz --quiet
import en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [56]:
# import docx2txt

# # replace following line with location of your .docx file
# MY_TEXT = docx2txt.process("/content/drive/MyDrive/Final_pipeline/NDA.docx")


# with open("/content/drive/MyDrive/Final_pipeline/Output.txt", "w") as text_file:
#     print(MY_TEXT, file=text_file)

In [57]:
#open text file in read mode
text_file = open("/content/drive/MyDrive/Final_pipeline/NDA.txt", "r")
 
#read whole file to a string
data = text_file.read()
 
#close file
text_file.close()

In [58]:
#def a function which can split the doc into sentences
def sentence_split(data):
    # nlp = spacy.load("en_core_web_sm")
    nlp = en_core_web_sm.load()
    
    #take unicode string  
    sentences = nlp(data)


    sentencess = list(sentences.sents)

    separated_sentences = []

    for i in range(len(sentencess)):
        sentence = sentencess[i].text
        #sentence_separation = re.split('\t|\n|\n\t', sentence)
        sentence = sentence.replace('\n', '')
        sentence = sentence.replace('\t', '')
        sentence = sentence.replace('\ufeff', '')
        sentence = sentence.replace('  ', '')
        sentence = sentence.replace('   ', '') #there are always multiple spaces, change them to single space universally
        sentences = re.split(r'\.(?=\s|$)', sentence)
        separated_sentences += sentences


    final_separated_sentences = [x for x in separated_sentences if x != '']
    return final_separated_sentences


splitted = sentence_split(data)


In [59]:
splitted

['Mutual Nondisclosure AgreementThis Mutual Nondisclosure Agreement ("Agreement") is made between Brex Inc',
 ' ("Brex") and ThoughtSpot Inc',
 ' ("Company")',
 'The parties may disclose confidential information as part of a potential business relationship or transaction of mutual interest ("Transaction") and agree to protect and maintain secrecy of the information disclosed in confidence for the Transaction',
 'The parties agree as follows:1',
 'Definitions',
 'In this Agreement: "Affiliate" means an entity that is directly or indirectly controls, is controlled by, or is under common control as a party, which shall include the power to direct or cause the direction of the management and policies of a person or an organization, whether by ownership of stock, voting rights, by contract, or otherwise; and "Confidential Information" is information disclosed by a party or its Affiliates or agents (collectively, the "Disclosing Party") to the other party or its Affiliates or agents (collect

In [60]:
#now define a function seperating the document into list of paragraphs

def par(splitted):
    ind = 0
    paragraphs = []
    splitting_index = []

    #get the index of splitting
    for i in range(len(splitted)):
        if len(splitted[i].split()) < 5:
            splitting_index.append(i)
    
    for j in range(len(splitting_index)):
        temp_para = ""
        while ind < splitting_index[j]:
            temp_para += splitted[ind]
            ind += 1
        paragraphs.append(temp_para)
    return [p for p in paragraphs if len(p.split()) > 5]

paragraphs = par(splitted)

In [61]:
# Name of the BERT model to use
model_name = 'bert-base-uncased'

# Max length of tokens
max_length = 100

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)


test_x = tokenizer(
    text=par(splitted),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

test_x

{'input_ids': <tf.Tensor: shape=(12, 100), dtype=int32, numpy=
array([[  101,  8203,  2512, ...,     0,     0,     0],
       [  101,  1006,  1000, ...,     0,     0,     0],
       [  101, 15182,  2378, ...,  2000,  1996,   102],
       ...,
       [  101,  7367, 26061, ...,     0,     0,     0],
       [  101,  3143,  3820, ...,     0,     0,     0],
       [  101,  2744, 15222, ...,     0,     0,     0]], dtype=int32)>}

In [62]:
#predict the possible value 
bert_result = bert_model.predict(test_x['input_ids'])

1/1 [==============================] - 16s 16s/step


In [63]:
#define a function, which can return the top 3 potential categories of a paragraph
#this method would return a dictionary: key is each paragraph, the corresponding values a the top 3 potential categories
target_names = ['DEF', 'EXP', 'GOV', 'REM', 'RIG', 'TER', 'WAR']

def par_cat(bert_result, paragraphs, target_names, topN):
    result = {}
    for i in range(len(bert_result["CAT"])):
        val = np.argsort(bert_result["CAT"][i])[::-1][:topN]
        result[paragraphs[i]] = [target_names[v] for v in val]
    return result


In [64]:
par_cat(bert_result, paragraphs, target_names, 3)

{'Mutual Nondisclosure AgreementThis Mutual Nondisclosure Agreement ("Agreement") is made between Brex Inc': ['RIG',
  'DEF',
  'EXP'],
 ' ("Company")The parties may disclose confidential information as part of a potential business relationship or transaction of mutual interest ("Transaction") and agree to protect and maintain secrecy of the information disclosed in confidence for the TransactionThe parties agree as follows:1': ['RIG',
  'DEF',
  'EXP'],
 'DefinitionsIn this Agreement: "Affiliate" means an entity that is directly or indirectly controls, is controlled by, or is under common control as a party, which shall include the power to direct or cause the direction of the management and policies of a person or an organization, whether by ownership of stock, voting rights, by contract, or otherwise; and "Confidential Information" is information disclosed by a party or its Affiliates or agents (collectively, the "Disclosing Party") to the other party or its Affiliates or agents (co